
# 測試檔

## 擷取人物單位

In [181]:
import pandas as pd
import json
import pathlib
import ast

In [182]:
news_df = pd.read_csv('result/total_opinions.csv')

In [183]:
news_df.head(1)

,title,article,url,company,category,reporter,space_index,datetime,year,month,day,hour,minute,article_coref,people_name,ner,topic,representative_docs,opinions_list,all_opinions_after_cluster
0,5共機擾台 運8反潛機連3天深入東南空域,"空軍今天晚間發布共機動態,5架共機(1架運8反潛機、1架空警500預警機、2架轟6、1架運8...",https://www.cna.com.tw/news/aipl/202205070219....,中央社,政治,游凱翔台北,"[92, 131, 244, 320, 450, 503, 643]",2022-05-07 19:40,2022,5,7,19,40,"空軍今天晚間發布共機動態,5架共機(1架運8反潛機、1架空警500預警機、2架轟6、1架運8...","['中央社', '國防部', '統合幕僚監部', '日本防衛省統合幕僚監部', '蘇紫雲',...","['台東', '遼寧號', '中央社', '台灣', '國防部', '日本', '台灣', ...",1,0,"[[[{'start': 503, 'end': 506, 'text': '蘇紫雲', '...","[{'person': '蘇紫雲', 'verb': '提到', 'opinion': '日..."


In [184]:
def download_pic(people_name):
    #Imports Packages
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from selenium import webdriver
    from webdriver_manager.chrome import ChromeDriverManager
    import urllib
    #Opens up web driver and goes to Google Images

    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
    driver.get('https://www.google.ca/imghp?hl=en&tab=ri&authuser=0&ogbl')
    box = driver.find_element_by_xpath('//*[@id="sbtc"]/div/div[2]/input')
    box.send_keys(people_name)
    box.send_keys(Keys.ENTER)
    img = driver.find_element_by_xpath('//*[@id="islrg"]/div[1]/div[1]/a[1]/div[1]/img')
    src = img.get_attribute('src')
    # download the image
    FILE_PATH = "pic/" + people_name + ".png"
    pathlib.Path(FILE_PATH).parent.mkdir(parents=True, exist_ok=True)
    urllib.request.urlretrieve(src, FILE_PATH)
    return FILE_PATH

In [185]:
person_unit_opinion_dict = {}

In [187]:
with open('person_unit_opinion.json',encoding="utf-8") as json_file:
    person_unit_opinion_dict = json.load(json_file)

In [188]:
def find_name_and_pic(name):
    for candidate in person_unit_opinion_dict.keys():
        # print(candidate)
        if name in candidate or candidate in name:
            return candidate, person_unit_opinion_dict[candidate]['pic_path']
    person_unit_opinion_dict[name] = {}
    person_unit_opinion_dict[name]['pic_path'] = download_pic(name)
    person_unit_opinion_dict[name]['opinions'] = []
    return name, person_unit_opinion_dict[name]['pic_path']

In [199]:
# create pic image link
for i in range(len(news_df)):
    opinions_list = ast.literal_eval(news_df.iloc[i]["opinions_list"])
    for opinions in opinions_list:
        for opinions_item in opinions[0]:
            label = opinions_item['labels']
            text = opinions_item['text']
            if label == ['person']:
                find_name_and_pic(text)

In [203]:
for i in person_unit_opinion_dict:
    person_unit_opinion_dict[i]['opinions'] = []

In [209]:
# change all opinions name
for i in range(len(news_df)):
    opinions_list = ast.literal_eval(news_df.iloc[i]["all_opinions_after_cluster"])
    if i == 38:
        print(news_df.iloc[i]['title'])
        print(opinions_list)
        print(news_df.iloc[i]["all_opinions_after_cluster"])
    for j in range(len(opinions_list)):
        # find person row
        # add opinions including time stamp
        name,_ = find_name_and_pic(opinions_list[j]['person'])
        if '小倉' in name:
            print(i, opinions_list[j]["person"], name)
        if opinions_list[j]["person"] != name:
            print(opinions_list[j]["person"], name)
            opinions_list[j]['person'] = name
    news_df.iloc[i]["all_opinions_after_cluster"] = opinions_list


5 小倉 小倉
13 小倉 小倉
13 小倉 小倉
15 小倉 小倉
「見證蔡英文非常有力的領導」 日本眾議員小倉將信:我深表敬意
[{'person': '小倉', 'verb': '表示', 'opinion': '此行來訪,因為有團員確診,因此無法與蔡英文見面,他深感遺憾,並感謝蔡英文前天在總統府熱情接待鈴木憲和(SUZUKINorikazu)眾議員等團員。', 'order': 1}, {'person': '小倉', 'verb': '表示', 'opinion': '6年來見證蔡英文非常有力的領導,台灣在政治安全面及經濟上都有長足發展,他要表達最深的敬意。', 'order': 0}, {'person': '小倉', 'verb': '提到', 'opinion': '中國近來不斷派出軍機入侵台灣的航空識別區,他也感到非常憂慮,台灣的安全受威脅,也會影響日本及日美同盟的安全。', 'order': 3}]
[{'person': '小倉', 'verb': '表示', 'opinion': '此行來訪,因為有團員確診,因此無法與蔡英文見面,他深感遺憾,並感謝蔡英文前天在總統府熱情接待鈴木憲和(SUZUKINorikazu)眾議員等團員。', 'order': 1}, {'person': '小倉', 'verb': '表示', 'opinion': '6年來見證蔡英文非常有力的領導,台灣在政治安全面及經濟上都有長足發展,他要表達最深的敬意。', 'order': 0}, {'person': '小倉', 'verb': '提到', 'opinion': '中國近來不斷派出軍機入侵台灣的航空識別區,他也感到非常憂慮,台灣的安全受威脅,也會影響日本及日美同盟的安全。', 'order': 3}]
38 小倉 小倉
38 小倉 小倉
38 小倉 小倉
74 小倉 小倉
74 小倉 小倉
74 小倉 小倉
82 小倉 小倉
82 小倉 小倉
93 小倉 小倉
93 小倉 小倉
139 小倉 小倉


In [191]:
# add person new opinions 
for i in range(len(news_df)):
    opinions_list = ast.literal_eval(news_df.iloc[i]["all_opinions_after_cluster"])
    for opinion in opinions_list:
        opinion['datetime'] = news_df.iloc[i]['datetime']
        del opinion['order']
        # find person row
        # add opinions including time stamp
        name,_ = find_name_and_pic(opinion['person'])
        person_unit_opinion_dict[name]['opinions'].append(opinion)


In [192]:
with open('person_unit_opinion.json', 'w', encoding='utf-8') as fp:
    json.dump(person_unit_opinion_dict, fp,  indent=4, ensure_ascii=False)


In [213]:
with open('person_unit_opinion.json',encoding="utf-8") as json_file:
    person_unit_opinion_dict = json.load(json_file)

In [ ]:
print(person_unit_opinion_dict)

In [218]:
for person_name in person_unit_opinion_dict:
    # print(person_name)
    temp_dic = person_unit_opinion_dict[person_name]
    PERSON_SLICE_FILE_PATH = 'person-slice-dataset/' + person_name + '.json'
    pathlib.Path(PERSON_SLICE_FILE_PATH).parent.mkdir(parents=True, exist_ok=True)
    with open(PERSON_SLICE_FILE_PATH, 'w', encoding='utf-8') as fp:
        json.dump(temp_dic, fp,  indent=4, ensure_ascii=False)
    break

## 情緒分析

In [2]:
import pandas as pd
import json
import pathlib
import ast

In [3]:
news_df = pd.read_csv('total_opinions.csv')

In [4]:
news_df

,title,article,url,company,category,reporter,space_index,datetime,year,month,day,hour,minute,article_coref,people_name,ner,topic,representative_docs,opinions_list,all_opinions_after_cluster
0,綠營大數據「疫情影響趨穩」 藍營聲量聚焦桃園內亂、徐巧芯違停,"在本土疫情升溫之際,在野黨持續針對快篩、用藥等議題砲轟執政團隊,不過在快篩劑鋪貨逐漸充足後,...",https://www.ettoday.net/news/20220522/2256707.htm,東森,政治,陶本和,"[118, 224, 339, 432, 491, 593, 707, 793, 871, ...",2022-05-22 21:55,2022,5,22,21,55,"在本土疫情升溫之際,在野黨持續針對快篩、用藥等議題砲轟執政團隊,不過在快篩劑鋪貨逐漸充足後,...","['張善政', '蔣萬安', '侯友宜', '康復者聯盟', '柯文哲', '徐巧芯', '...","['桃園', '張善政', '台北市', '蔣萬安', '侯友宜', '康復者聯盟', '柯...",0,0,"[[[{'start': 876, 'end': 879, 'text': '侯友宜', '...","[{'person': '侯友宜', 'verb': '指出', 'opinion': '他..."
1,"中央不幫忙還來「拆臺」打擊士氣,怪不得侯友宜「火大」?","總統府發言人張惇涵今天(5/22)表示,蔡英文5月19日曾打電話給新北醫師公會,協調醫護人力...",https://www.ettoday.net/news/20220522/2256710.htm,東森,雲論,韋安,"[77, 143, 200, 213, 214, 233, 258]",2022-05-22 21:52,2022,5,22,21,52,"總統府發言人張惇涵今天(5/22)表示,蔡英文5月19日曾打電話給新北醫師公會,協調醫護人力...","['蔡英文', '小英', '新北醫師公會', '侯友宜', '雙北', '王必勝', '總...","['蔡英文', '新北市', '新北', '小英', '新北醫師公會', '侯友宜', '北...",0,0,"[[[{'start': 6, 'end': 9, 'text': '張惇涵', 'labe...","[{'person': '張惇涵', 'verb': '表示', 'opinion': '蔡..."
2,柯文哲自曝確診過程 「趁著這機會檢視防疫體系如何優化」,"台北市長柯文哲22日上午因為咳嗽、身體不適,因此自行做快篩呈現陽性反應,並緊急到聯合醫院仁愛...",https://www.ettoday.net/news/20220522/2256709.htm,東森,政治,杜冠霖,"[134, 291, 391, 493, 547, 599]",2022-05-22 21:51,2022,5,22,21,51,"台北市長柯文哲22日上午因為咳嗽、身體不適,因此自行做快篩呈現陽性反應,並緊急到聯合醫院仁愛...","['佩琪', '柯文哲', '仁愛院區', '彭振聲', '蔡炳坤', '台北市長', '黃...","['佩琪', '柯文哲', '柯', '仁愛院區', '臺灣', '彭振聲', '柯文哲',...",0,0,"[[[{'start': 86, 'end': 89, 'text': '柯文哲', 'la...","[{'person': '柯文哲', 'verb': '表示', 'opinion': '想..."
3,美國指台灣不會列入印太經濟架構 國安人士拋「觀察重點」,"美國白宮國家安全顧問蘇利文(JakeSullivan)22日受訪時,預先透露台灣將不會成為「...",https://www.ettoday.net/news/20220522/2256714.htm,東森,政治,陶本和,"[18, 124, 142, 232, 309, 356, 367, 404, 457, 5...",2022-05-22 21:47,2022,5,22,21,47,"美國白宮國家安全顧問蘇利文(JakeSullivan)22日受訪時,預先透露台灣將不會成為「...","['蘇利文', '鄧振中', 'IPEF', '美國白宮', '李淳即', '戴琪', 'U...","['國安', '白宮', '台', '台灣', '美方', '蘇利文', '台灣', '台'...",1,0,"[[[{'start': 54, 'end': 58, 'text': 'IPEF', 'l...","[{'person': 'IPEF', 'verb': '強調', 'opinion': '..."
4,美指台灣不會列入印太經濟架構 外交部稱遺憾:未來仍會積極爭取,"美國白宮國家安全顧問蘇利文(JakeSullivan)22日受訪時,預先透露台灣將不會成為「...",https://www.ettoday.net/news/20220522/2256704.htm,東森,政治,杜冠霖,"[18, 131, 224, 237, 238, 244, 316, 434, 488]",2022-05-22 21:26,2022,5,22,21,26,"美國白宮國家安全顧問蘇利文(JakeSullivan)22日受訪時,預先透露台灣將不會成為「...","['外交部', 'IPEF', '戴琪', '鄧振中', '蘇利文', '美國白宮', '行...","['台灣', '台', '外交部', '印太', 'IPEF', '台灣', '美', '台...",1,0,"[[[{'start': 54, 'end': 58, 'text': 'IPEF', 'l...","[{'person': 'IPEF', 'verb': '強調', 'opinion': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,靠曾是黑五類的他牽線!安倍避疫情高峰料7月訪台 適逢李登輝逝世2周年,"前日本首相安倍晉三日前表達訪台的強烈意願,但礙於疫情等因素延宕。據知情人士透露,日方透過日本...",https://www.storm.mg/article/4343710,風傳媒,"政治,調查,VIP,政經解密",林庭瑤,"[138, 260, 386, 511, 681, 874]",2022-05-22 08:30,2022,5,22,8,30,"前日本首相安倍晉三日前表達訪台的強烈意願,但礙於疫情等因素延宕。據知情人士透露,日方透過日本...","['李登輝', '自民黨', '產經新聞', '陳唐山', '矢板明夫', '新華社', '...","['李登輝', '自民黨', '北京', '日本', '華', '中共', '台', '台灣...",1,0,"[[[{'start': 462, 'end': 466, 'text': '產經新聞', ...","[{'person': '產經新聞', 'verb': '說', 'opinion': '著..."
135,徵召張善政犯眾怒!朱立倫負評激增 網路輿情「反藍黨中央」聲量最高,"桃園市選戰年底登場,國民黨主席朱立倫本周在中常會無預警宣布,將徵召無黨籍的前行政院長張善政,...",https://www.storm.mg/article/4344193,風傳媒,政治,QuickseeK快析輿情資料庫,"[174, 477, 839, 1100]",2022-05-22 08:20,2022,5,22,8,20,"桃園市選戰年底登場,國民黨主席朱立倫本周在中常會無預警宣布,將徵召無黨籍的前行政院長張善政,...","['國民黨', '羅智強', '朱立倫', '張善政', '行政院長', '呂玉玲', '邱...","['國民黨', '羅智強', '國民黨', '國民黨', '朱立倫', '朱立倫', '羅智...",0,0,"[[[{'start': 15, 'end': 18, 'text': '朱立倫', 'la...","[{'person': '朱立倫', 'verb': '宣布', 'opinion': '將..."
136,蘇貞昌大讚蔡英文執政6年政績 施正鋒酸:儼然狄仁傑之於武則天,"總統蔡英文屆滿6周年,行政院長蘇貞昌日前曾說,蔡英文任內國家主權、民眾安全、經濟發展、國民照...",https://www.storm.mg/article/4344222,風傳媒,政治,羅立邦,"[86, 208, 283, 379, 488, 688]",2022-05-22 08:10,2022,5,22,8,10,"總統蔡英文屆滿6周年,行政院長蘇貞昌日前曾說,蔡英文任內國家主權、民眾安全、經濟發展、國民照...","['施正鋒', '蔡英文', '衛福部長', '台灣民意基金會', '老王', '東華大學'...","['施正鋒', '蔡英文', '衛福部長', '台灣民意基金會', '施正鋒', '老王',.

In [5]:
# change all opinions name
for i in range(len(news_df)):
    opinions_list = ast.literal_eval(news_df.iloc[i]["all_opinions_after_cluster"])
    for j in range(len(opinions_list)):
        # find person row
        # add opinions including time stamp
        # if '小倉' in name:
        #     print(i, opinions_list[j]["person"], name)
        # if opinions_list[j]["person"] != name:
        #     print(opinions_list[j]["person"], name)
        #     opinions_list[j]['person'] = name
        print(opinions_list[j]['opinion'])
    # news_df.iloc[i]["all_opinions_after_cluster"] = opinions_list


他的整體聲量高點卻是落在4月19日(2.4萬),侯友宜因新北兩歲童等不到救護車離世一案,遭到質疑。
蔡英文5月19日曾打電話給新北醫師公會,協調醫護人力。
雙北叫不動醫院?暗指地方防疫不力?新北自4月11日,至5月21日,累計照護達29萬9040人,解隔康復達17萬8192人。
「我知道總統太忙了,我會一肩承擔新北市的大小事,一定守好新北市!」。
想趁這個機會,向全臺灣所有的醫護人員表達最深的敬意。
回家經過七個小時,下午將近六點自己收到了聯醫的確診通知,自我隔離的過程中,也在想如何避免傳染給家人,「尤其是佩琪每天要去上班,一旦她也感染請假,恐怕會再增加其他同事的負擔。」
要趁這次機會,「用確診者的角度,檢視目前防疫體系可以如何優化,讓民眾得到更好的照顧。」
美方期盼深化與台灣的經貿夥伴關係,以及台美雙邊關係的強化將與IPEF同時並進。
短期而言,鄧振中與戴琪會面的內容及後續台美貿易進展更值得關注,尤其是美方罕見地釋放出可望數週內會商,共尋具體途徑來增進雙方合作,事實上是明顯進展,對於我方推動洽簽台美雙邊貿易協定(BTA)的前置階段,也可謂是階段性地正面推進。
鄧振中與戴琪的會面,確定台美彼此為「優先貿易夥伴關係」(prioritytradingpartners)之關係,無論台灣是否在首輪加入,台美關係仍持續緊密。
美方期盼深化與台灣的經貿夥伴關係,以及台美雙邊關係的強化將與IPEF同時並進。
另一方面,如同蘇利文國安顧問所言,台美正在尋求建構深化雙邊經濟夥伴關係的機制。
對於我國未獲納入IPEF首輪名單感到遺憾。
遺憾,但未來仍會積極爭取參與。
民進黨雲林縣黨部主委選戰已殺紅眼,話一說出不久即傳出由立法委員劉建國推派出選的參選人沈健興退選,全力支持曾擔任立委蘇治芬助理主任的林金忠同額競選,今(22)日結果出爐,主任委員由林金忠以3906票當選。
今天縣黨部所設的十個投開票所選出主委林金忠、9名全國黨代表黃清財、蔡孟真、林淑靜、江文登、莊志勤、廖秋萍、蔡明宏、陳建名、呂順賢及46名縣代表;執行委員及評議委員改選日期將6月26日召開雲林縣代表大會進行改組。
退選,但他照樣橫掃近8000票,只輸給何博文不到3000票。
余天大哥德高望重,和他同時登記主委,從來就不是競爭,而是走一個必須的民主程序而已,「感謝余天大哥,衷心祝福余天大哥家庭平安順心」。
退選的余天拿下774

In [7]:
negative_list = []
f = open('sentiment_dict/NTUSD_negative_unicode.txt',encoding='utf16')
for line in f.readlines():
    negative_list.append(line.replace('\n',''))

positive_list = []
f = open('sentiment_dict/NTUSD_positive_unicode.txt',encoding='utf16')
for line in f.readlines():
    positive_list.append(line.replace('\n',''))

In [12]:
print(negative_list)
# print(len(positive_list))

['一下子爆發', '一下子爆發的一連串', '一巴掌', '一再', '一再叮囑', '一拳', '一般殺人罪', '一陣狂風', '一陣緊張', '一掌', '一團糟', '一摑', '一點點', '一蹶不振', '人事不省', '人為', '人為的', '入迷', '入迷的', '入迷的人', '刀刃', '刁難', '力盡', '匕首', '下地獄', '下垂', '下垂度', '下流', '下流的', '下降', '下陷', '下等', '下等的', '下跌', '下獄', '下層社會', '下暴雨', '上當', '上鉤', '上癮', '上癮者', '久坐', '久坐的', '久經', '久經世故', '久經世故的', '乞丐', '乞求', '乞討', '乞得', '亡命', '亡命者', '口吃', '口角', '口渴', '口渴的', '土匪', '士氣低落', '大叫', '大叫大嚷著要求', '大失敗', '大吃一驚', '大吃一驚的', '大批出沒', '大災難', '大雨', '大洪水', '大恐慌', '大浪', '大破壞', '大屠殺', '大量殺戮', '大量噴出', '大傷', '大意', '大意的', '大話', '大禍', '大錯', '大聲', '大聲的要求', '大聲要求', '大聲要求的', '大膽', '大膽的', '大嚷', '大變革', '大變動', '大驚小怪', '女巫', '子彈', '小人', '小心', '小卒', '小氣', '小氣的', '小偷', '小規模戰鬥', '小傢伙', '小搗蛋', '小衝突', '小題大作', '工作過度', '工作過度的', '己見', '已過盛年', '已過盛年的', '干涉', '干擾', '干擾者', '不一致', '不上算的', '不大', '不大可能', '不大可能之事', '不小心', '不小心的', '不公平', '不公平的', '不公正', '不切實際', '不切實際的', '不友好', '不友好的', '不友善', '不友善地', '不友善的', '不及格', '不太', '不太可能的', '不引人注意', '不引人注意的', '不毛', '不毛的', '不充分', '不充分的', '不充足', '不充足的', '不出場', '不加', '不加思考',

In [9]:
for i in range(len(news_df)):
    opinions_list = ast.literal_eval(news_df.iloc[i]["all_opinions_after_cluster"])
    for j in range(len(opinions_list)):
        n_score = 0
        p_score = 0
        for nw in negative_list:
            if nw in opinions_list[j]['opinion']:
                n_score += 1
        for pw in positive_list:
            if pw in opinions_list[j]['opinion']:
                p_score += 1
        opinions_list[j]['negative'] = n_score
        opinions_list[j]['positive'] = p_score
        print(opinions_list[j]['opinion'],"n_score = ", n_score, "p_score = ", p_score)

他的整體聲量高點卻是落在4月19日(2.4萬),侯友宜因新北兩歲童等不到救護車離世一案,遭到質疑。 n_score =  5 p_score =  2
蔡英文5月19日曾打電話給新北醫師公會,協調醫護人力。 n_score =  2 p_score =  3
雙北叫不動醫院?暗指地方防疫不力?新北自4月11日,至5月21日,累計照護達29萬9040人,解隔康復達17萬8192人。 n_score =  3 p_score =  2
「我知道總統太忙了,我會一肩承擔新北市的大小事,一定守好新北市!」。 n_score =  2 p_score =  3
想趁這個機會,向全臺灣所有的醫護人員表達最深的敬意。 n_score =  2 p_score =  3
回家經過七個小時,下午將近六點自己收到了聯醫的確診通知,自我隔離的過程中,也在想如何避免傳染給家人,「尤其是佩琪每天要去上班,一旦她也感染請假,恐怕會再增加其他同事的負擔。」 n_score =  10 p_score =  6
要趁這次機會,「用確診者的角度,檢視目前防疫體系可以如何優化,讓民眾得到更好的照顧。」 n_score =  2 p_score =  4
美方期盼深化與台灣的經貿夥伴關係,以及台美雙邊關係的強化將與IPEF同時並進。 n_score =  2 p_score =  4
短期而言,鄧振中與戴琪會面的內容及後續台美貿易進展更值得關注,尤其是美方罕見地釋放出可望數週內會商,共尋具體途徑來增進雙方合作,事實上是明顯進展,對於我方推動洽簽台美雙邊貿易協定(BTA)的前置階段,也可謂是階段性地正面推進。 n_score =  3 p_score =  10
鄧振中與戴琪的會面,確定台美彼此為「優先貿易夥伴關係」(prioritytradingpartners)之關係,無論台灣是否在首輪加入,台美關係仍持續緊密。 n_score =  2 p_score =  4
美方期盼深化與台灣的經貿夥伴關係,以及台美雙邊關係的強化將與IPEF同時並進。 n_score =  2 p_score =  4
另一方面,如同蘇利文國安顧問所言,台美正在尋求建構深化雙邊經濟夥伴關係的機制。 n_score =  2 p_score =  3
對於我國未獲納入IPEF首輪名單感到遺憾。 n_score =  6 p_sc